In [ ]:
from itertools import product

def get_codon_table(table_number):
    with open('translation_tables.txt', 'r') as f:
        s = f.read().partition('==== Table '+str(table_number)+' ====')[2].partition('=================')[0]

    codon_table = {}
    for a in s.split('\n'):
        codones_string = a.split(' ')
        codones_string = ' '.join(codones_string).split()
        for ch in range(len(codones_string)):
            if codones_string[ch].isupper() and (len(codones_string[ch]) == 3):
                if ch+3 in range(len(codones_string)):
                    if codones_string[ch+3] == 'i':
                        codon_table[codones_string[ch]] = (codones_string[ch+1], 
                                                           codones_string[ch+2], 
                                                           codones_string[ch+3])
                        ch += 2
                    else:
                        codon_table[codones_string[ch]] = (codones_string[ch+1], 
                                                           codones_string[ch+2])
                        ch += 1
                else:
                    codon_table[codones_string[ch]] = (codones_string[ch+1], 
                                                       codones_string[ch+2])
    return codon_table

def translate(seq, codon_table): 
    seq = seq.replace('U', 'T')
    protein = ''
    if len(seq)%3 == 0: 
        for i in range(0, len(seq), 3): 
            codon = seq[i:i + 3] 
            protein+= codon_table[codon][0]
    else:
        print('Number of nucleotides does not divide into three')
    return protein

def get_codon_variations(codon, codon_table):
    codon = codon.replace('U', 'T')
    amino_acid = translate(codon, codon_table)
    
    codons_all = []
    for c in product('ACGT', repeat = 3):
        codons_all.append(''.join(c))
    
    variations_raw = []
    for ex in codons_all:
        diff = lambda l1,l2: len([x for ind, x in enumerate(l1) if l1[ind] != l2[ind]])
        variations_raw.append([ex,diff(codon, ex)])
       
    variations_sorted = sorted(variations_raw, key=lambda x:x[1])
    
    variations = []
    for var in variations_sorted:
        amino_acid_m = translate(var[0], codon_table) 
        if (amino_acid_m not in [i[0] for i in variations]) and (amino_acid_m != '*'):
            variations.append([amino_acid_m, var[1]])
    
    variations_count = {
        0 : [i[1] for i in variations].count(0),
        1 : [i[1] for i in variations].count(1),
        2 : [i[1] for i in variations].count(2),
        3 : [i[1] for i in variations].count(3)
    }
        
    return variations_count

def get_sequence_number(sequence, mutate_first_codon, max_nmut, codon_table):
    a = 0
    implement = {}
    implement[(0,0)] = 1
    mutations = 0
    current_codon = 1
    length = len(translate(sequence, codon_table))

    while a <= len(sequence)-3:
        max_mutations = 0 if current_codon == 1 and not mutate_first_codon else 3
        prev_mutations = mutations
        mutations += max_mutations
        current_codon_variations = get_codon_variations(sequence[a:a+3], codon_table)

        nmut = 0
        while nmut <= mutations:
            extra_m = 0
            implement[(nmut, current_codon)] = 0
            while extra_m <= max_mutations:
                prev_nmut = nmut - extra_m
                if (prev_nmut < 0) or (prev_nmut > prev_mutations):
                    extra_m += 1
                    continue
                implement[(nmut, current_codon)] += implement[(prev_nmut, current_codon - 1)] * current_codon_variations[extra_m]
                extra_m += 1
            nmut += 1

        current_codon += 1
        a += 3

    integral = 0
    integral_dict = {}
    nmut = 0
    while nmut <= mutations:
        integral += implement[(nmut, length)]
        integral_dict[(nmut, length)] = integral
        nmut += 1

    nmut = 0
    while (nmut <= max_nmut) and (nmut <= max(implement.keys())[0]):
        print('{0} {1} {2}'.format(nmut, implement[(nmut,length)], integral_dict[(nmut,length)]))
        nmut += 1

        
mutate_first_codon = 0
gfp = 'AUGUCG'
gfp = 'ATGGTGAGCAAGGGCGAGGAGCTGTTCACCGGGGTGGTGCCCATCCTGGTCGAGCTGGACGGCGACGTAAACGGCCACAAGTTCAGCGTGTCCGGCGAGGGCGAGGGCGATGCCACCTACGGCAAGCTGACCCTGAAGTTCATCTGCACCACCGGCAAGCTGCCCGTGCCCTGGCCCACCCTCGTGACCACCCTGACCTACGGCGTGCAGTGCTTCAGCCGCTACCCCGACCACATGAAGCAGCACGACTTCTTCAAGTCCGCCATGCCCGAAGGCTACGTCCAGGAGCGCACCATCTTCTTCAAGGACGACGGCAACTACAAGACCCGCGCCGAGGTGAAGTTCGAGGGCGACACCCTGGTGAACCGCATCGAGCTGAAGGGCATCGACTTCAAGGAGGACGGCAACATCCTGGGGCACAAGCTGGAGTACAACTACAACAGCCACAACGTCTATATCATGGCCGACAAGCAGAAGAACGGCATCAAGGTGAACTTCAAGATCCGCCACAACATCGAGGACGGCAGCGTGCAGCTCGCCGACCACTACCAGCAGAACACCCCCATCGGCGACGGCCCCGTGCTGCTGCCCGACAACCACTACCTGAGCACCCAGTCCGCCCTGAGCAAAGACCCCAACGAGAAGCGCGATCACATGGTCCTGCTGGAGTTCGTGACCGCCGCCGGGATCACTCTCGGCATGGACGAGCTGTACAAG'
max_nmut = 30
codon_table = get_codon_table(1)

get_sequence_number(gfp, mutate_first_codon, max_nmut, codon_table)

In [6]:
import sequence_number as sn

mutate_first_codon = 0
gfp = 'ATGGTGAGCAAGGGCGAGGAGCTGTTCACCGGGGTGGTGCCCATCCTGGTCGAGCTGGACGGCGACGTAAACGGCCACAAGTTCAGCGTGTCCGGCGAGGGCGAGGGCGATGCCACCTACGGCAAGCTGACCCTGAAGTTCATCTGCACCACCGGCAAGCTGCCCGTGCCCTGGCCCACCCTCGTGACCACCCTGACCTACGGCGTGCAGTGCTTCAGCCGCTACCCCGACCACATGAAGCAGCACGACTTCTTCAAGTCCGCCATGCCCGAAGGCTACGTCCAGGAGCGCACCATCTTCTTCAAGGACGACGGCAACTACAAGACCCGCGCCGAGGTGAAGTTCGAGGGCGACACCCTGGTGAACCGCATCGAGCTGAAGGGCATCGACTTCAAGGAGGACGGCAACATCCTGGGGCACAAGCTGGAGTACAACTACAACAGCCACAACGTCTATATCATGGCCGACAAGCAGAAGAACGGCATCAAGGTGAACTTCAAGATCCGCCACAACATCGAGGACGGCAGCGTGCAGCTCGCCGACCACTACCAGCAGAACACCCCCATCGGCGACGGCCCCGTGCTGCTGCCCGACAACCACTACCTGAGCACCCAGTCCGCCCTGAGCAAAGACCCCAACGAGAAGCGCGATCACATGGTCCTGCTGGAGTTCGTGACCGCCGCCGGGATCACTCTCGGCATGGACGAGCTGTACAAG'
gfp = 'AUGUCG'
max_nmut = 30
codon_table = sn.get_codon_table(1)
s = sn.get_sequence_number(gfp, mutate_first_codon, max_nmut, codon_table)
s

['0\t1\t1', '1\t5\t6', '2\t10\t16', '3\t4\t20']

In [3]:
str(s)

"['0\\t1\\t1', '1\\t1428\\t1429', '2\\t1017668\\t1019097', '3\\t482582301\\t483601398', '4\\t171307137200\\t171790738598', '5\\t48556340988458\\t48728131727056', '6\\t11447497145830389\\t11496225277557445', '7\\t2308886493724271456\\t2320382719001828901', '8\\t406701526641104738030\\t409021909360106566931', '9\\t63557749318116382648317\\t63966771227476489215248', '10\\t8922254634669921993440048\\t8986221405897398482655296', '11\\t1136469514130557582186111816\\t1145455735536454980668767112', '12\\t132440566209385992581792042145\\t133586021944922447562460809257', '13\\t14219674075969977032228147237232\\t14353260097914899479790608046489', '14\\t1414946129715544493473550259433238\\t1429299389813459392953340867479727', '15\\t131157196445243841533581922534474757\\t132586495835057300926535263401954484', '16\\t11375739527095640648017306077459468671\\t11508326022930697948943841340861423155', '17\\t926832101364434814976939862540519002112\\t938340427387365512925883703881380425267', '18\\t71180427